<a href="https://colab.research.google.com/github/mrarat76/Tensorflow_Practises/blob/main/NLP_Practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## recurrent neural network

## LSTM rnn hücrelerine denir "tanh" non-linearity eklemek için kullnaırız


## Outputta sigmodi kullanıyoruz


NLP'nin amacı bilgiyi doğal dilden elde etmektir( metini veya konuşmayı sıralamak gibi)

NLP genel manada seq2seq problemlerine odaklanır.


**Seq2seq**, kısaltması "sequence to sequence" (diziden diziye) anlamına gelir ve genellikle doğal dil işleme (NLP) ve makine öğrenmesinde kullanılan bir model türüdür. Bu model, bir girdi dizisini (örneğin bir cümle veya kelime dizisi) alır ve başka bir diziye (genellikle başka bir dilde bir cümle veya metin) çevirir. Seq2seq modelleri, özellikle makine çevirisi, otomatik özetleme, soru-cevap sistemleri ve sohbet robotları gibi alanlarda yaygın olarak kullanılır.

Seq2seq modeli tipik olarak iki ana bileşenden oluşur:

Encoder (Kodlayıcı): Girdi dizisini alır ve bu diziyi sabit boyutlu bir vektör temsiline dönüştürür. Bu vektör, girdi dizisinin bir tür özeti olarak düşünülebilir ve genellikle diziye ait önemli bilgileri içerir.

Decoder (Çözücü): Encoder tarafından oluşturulan sabit boyutlu vektörü alır ve hedef diziye dönüştürür. Çözücü, bu vektörü kullanarak adım adım çıktı dizisini üretir.

Bu modeller genellikle tekrarlayan sinir ağları (RNN), uzun kısa süreli bellek ağları (LSTM) veya gated recurrent units (GRU) gibi yapıları kullanır. Son yıllarda, daha gelişmiş seq2seq modelleri, dikkat mekanizmaları (attention mechanisms) ve Transformer mimarileri gibi teknikleri kullanarak daha iyi performans sergilemiştir. Dikkat mekanizması, modelin girdi dizisinin hangi bölümlerine odaklanması gerektiğini daha iyi anlamasını sağlar ve böylece daha doğru çeviriler yapılmasına olanak tanır.







In [2]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-03-10 16:02:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-03-10 16:02:25 (45.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-03-10 16:02:30--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 74.125.23.207, 74.125.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   855KB/s    in 0.7s    

2024-03-10 16:02:31 (855 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
### Visualizing a text dataset

import pandas as pd

train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

train_df.head()



,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
## Dataframi shuffellama

train_df_shuffled =train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()



,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
## How many total samples?


len(train_df), len(test_df)

(7613, 3263)

In [12]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target =row
  print(f"Target:{target}", "(real_diaster)" if target > 0 else "(not real diaster)")
  print(f"Text\n{text}\n")
  print("---\n")



Target:0 (not real diaster)
Text
I honestly wonder what it is that I had to do so wrong to lose everyone.

---

Target:0 (not real diaster)
Text
The feeling of lonelyness has sunk into me strange depressing feeling.

---

Target:0 (not real diaster)
Text
I have my final tomorrow ?????? . Ish is stressful  but I'll survive

---

Target:0 (not real diaster)
Text
Why does it say Silas sliced in that headlinelike someone chopped him up like a piece of cabbage????????????????????????  #GH

---

Target:1 (real_diaster)
Text
@rahulkanwal why  Jammu is soft target for terrorist

---



In [13]:

from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [14]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [17]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [18]:
max_vocab_lenght= 10000
max_length=15

text_vectorizer= TextVectorization(max_tokens= max_vocab_lenght,
                                   output_mode="int",
                                   output_sequence_length= max_length)


In [19]:
## Fit the text vectorizer

text_vectorizer.adapt(train_sentences)


sample_sentence="There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Random sentence choose


random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence}\
\n\nVectorized version:")

text_vectorizer([random_sentence])

Original text: 
 KATUNews: #SR14 remains closed as brush fire burns 1700 acres: http://t.co/QposKp3MWj #LiveOnK2 http://t.co/mTQjsvupwy

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[5286, 8167, 1778,  959,   26, 4057,   42, 1887,    1, 1722,    1,
        5211,    1,    0,    0]])>

In [21]:
## Get the unique words in the vocab


words_in_vocab =text_vectorizer.get_vocabulary()

top_5_words = words_in_vocab[5:]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'youtube', 'dead', 'two', 'families', '3', 'train', 'full', 'being', 'war', 'many', 'today', 'think', 'only',

In [22]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

Yukarıda yaptıklarımız tokenization layer oluyor.


## Embedding ve Tokenization FARKI


Embedding ve tokenization, doğal dil işleme (NLP) alanında kullanılan iki farklı teknikdir. İkisi de metin verilerini makine öğrenimi modellerine veya yapay zeka sistemlerine giriş olarak kullanılabilir. Ancak işlevleri ve kullanımları farklıdır.

**Tokenization**:
Tokenization, metin verilerini belirli bir format veya dil bilgisine göre parçalara ayırma işlemidir. Bu parçalar "token" olarak adlandırılır. Tokenization sırasında, metin cümleler veya kelimeler gibi daha küçük parçalara bölünür. Örneğin, bir cümle cümlelerine veya kelimelerine ayrılabilir.
Tokenization, metnin yapısını anlamak, kelime sıklığı sayımı yapmak, dil modelleri oluşturmak veya bir metin veri kümesini işlemek için kullanılır. Örnek olarak, bir cümleyi tokenize ederken, "Bu bir örnek cümledir." gibi bir giriş "Bu", "bir", "örnek", "cümledir" gibi ayrı ayrı parçalara bölünebilir.

**Embedding**:
Embedding, metin veya kategorik verileri makine öğrenimi modellerine veya yapay zeka sistemlerine giriş olarak temsil etmek için kullanılan bir tekniktir. Embedding, tokenization'ın ötesine geçerek, kelime veya kategori gibi girişlerin bir vektör uzayında temsil edilmesini sağlar. Bu vektör temsilleri, kelimenin veya kategorinin anlamsal benzerliklerini veya ilişkilerini yansıtacak şekilde öğrenilir.
Özellikle kelime gömme (word embedding) sıkça kullanılan bir embedding türüdür. Burada, her kelime bir vektörle temsil edilir ve bu vektörler, genellikle kelimenin anlamını yansıtacak şekilde eğitilir. Örneğin, "kral" ve "kraliçe" kelimeleri arasındaki ilişkiyi temsil eden vektörler birbirlerine daha yakın olacaktır.

Sonuç olarak, tokenization, metin verilerini işlemek ve parçalara ayırmak için kullanılırken, embedding, metin verilerini sayısal temsillerle temsil etmek ve anlamsal ilişkileri yansıtmak için kullanılır. Birçok durumda, öncelikle metin verileri tokenize edilir ve ardından bu tokenler embedding teknikleri kullanılarak sayısal temsillerle değiştirilir. Bu, makine öğrenimi modellerinin metin verilerini anlaması ve işlemesi için önemli bir adımdır.

In [23]:
### Embedding layer
tf.random.set_seed(42)

max_vocab_length = 10000  # Example value, replace with the actual size of your vocabulary

from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding


In [24]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Killing Black Babies at Planned Parenthood where's the demonstrations looting of PP burning down the buildings Black Babies Lives Matter      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0077737 , -0.03476388,  0.0174867 , ..., -0.01193758,
          0.02319798,  0.0043537 ],
        [ 0.00973732, -0.02333597, -0.006516  , ..., -0.01825236,
          0.03037998,  0.02352151],
        [-0.00644572, -0.04884225, -0.01022481, ..., -0.03947739,
          0.01770595,  0.0271737 ],
        ...,
        [-0.0423654 , -0.01330614, -0.03133594, ...,  0.0342023 ,
         -0.00227011, -0.01031109],
        [ 0.00689237, -0.02481714,  0.01971697, ..., -0.00210954,
          0.02170194,  0.01995095],
        [ 0.03873457,  0.0433658 ,  0.03647766, ..., -0.02003136,
         -0.03525887,  0.01171626]]], dtype=float32)>

In [25]:
sample_embed[0][0]


<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.0077737 , -0.03476388,  0.0174867 ,  0.04237205,  0.02693233,
       -0.04004189,  0.01401256, -0.00329285, -0.02026932,  0.02476465,
       -0.01237895, -0.01307858, -0.03852428, -0.00749549, -0.01663154,
        0.00548518,  0.04014316, -0.02715404,  0.03626496,  0.046301  ,
       -0.00171707,  0.04015705,  0.04227242,  0.03913656, -0.03297552,
        0.02873844,  0.04708539,  0.01046395, -0.04873145,  0.01925225,
        0.00368967,  0.04739388,  0.03275077, -0.0324529 ,  0.03536263,
       -0.01232357,  0.01526761, -0.03635786, -0.02535859,  0.00629757,
        0.01776352,  0.01353811,  0.04291191,  0.03689443,  0.04642412,
       -0.01707525, -0.01554103,  0.04809194,  0.04851294,  0.04316706,
        0.03210518,  0.01077477,  0.04111272, -0.04122711, -0.01609017,
       -0.00784174,  0.00490875, -0.0283637 , -0.01120373,  0.03403104,
        0.01820823,  0.01832088, -0.03989033, -0.01559234, -0.02336699,
       -0.029905

In [26]:
## Text verisini modelleme (running a series of experiments)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [30]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1

In [31]:
from helper_functions import create_tensorboard_callback

SAVE_DIR="model_logs"

In [32]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens) Her bir token için tahminleme yapar eğer global average pool yapmaz isek.
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [33]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [35]:
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20240310-160235
Epoch 1/5
215/215 [==============================] - 10s 31ms/step - loss: 0.6081 - accuracy: 0.6948 - val_loss: 0.5356 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 6s 29ms/step - loss: 0.4403 - accuracy: 0.8194 - val_loss: 0.4697 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.3456 - accuracy: 0.8613 - val_loss: 0.4598 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.2840 - accuracy: 0.8921 - val_loss: 0.4652 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2371 - accuracy: 0.9121 - val_loss: 0.4779 - val_accuracy: 0.7861


In [36]:
model_1.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 2ms/step - loss: 0.4779 - accuracy: 0.7861


[0.47786661982536316, 0.7860892415046692]

In [37]:
embedding.weights


[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.01692829,  0.01096282,  0.04647324, ..., -0.00040925,
          0.0103692 , -0.01446817],
        [ 0.01794673, -0.00074595,  0.01143022, ..., -0.03659263,
          0.0038916 , -0.04195898],
        [ 0.05798328,  0.06116285,  0.0552786 , ..., -0.03765529,
         -0.05672168, -0.00586684],
        ...,
        [-0.03546869, -0.00080062,  0.0081836 , ...,  0.01381333,
          0.0147727 , -0.02697916],
        [ 0.07818292,  0.05341816,  0.04668284, ..., -0.04992466,
         -0.08295695, -0.03535902],
        [ 0.08034026,  0.05145524,  0.02719898, ..., -0.03712629,
         -0.08278388, -0.01702061]], dtype=float32)>]

In [38]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)


(10000, 128)


In [39]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.40948546],
       [0.7450638 ],
       [0.9978547 ],
       [0.10921004],
       [0.10459675],
       [0.93793875],
       [0.91359055],
       [0.99265665],
       [0.96865016],
       [0.26476055]], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [42]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [43]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.61, Difference: -0.66
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


In [44]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.40948546],
       [0.7450638 ],
       [0.9978547 ],
       [0.10921004],
       [0.10459675],
       [0.93793875],
       [0.91359055],
       [0.99265665],
       [0.96865016],
       [0.26476055]], dtype=float32)

In [45]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [46]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results


{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [47]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [48]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.61, Difference: -0.66
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


# **Embeddingleri görselleştirme**

In [49]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [50]:
model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [51]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [52]:
embed_weights


##https://www.tensorflow.org/text/guide/word_embeddings?hl=tr

array([[-0.01692829,  0.01096282,  0.04647324, ..., -0.00040925,
         0.0103692 , -0.01446817],
       [ 0.01794673, -0.00074595,  0.01143022, ..., -0.03659263,
         0.0038916 , -0.04195898],
       [ 0.05798328,  0.06116285,  0.0552786 , ..., -0.03765529,
        -0.05672168, -0.00586684],
       ...,
       [-0.03546869, -0.00080062,  0.0081836 , ...,  0.01381333,
         0.0147727 , -0.02697916],
       [ 0.07818292,  0.05341816,  0.04668284, ..., -0.04992466,
        -0.08295695, -0.03535902],
       [ 0.08034026,  0.05145524,  0.02719898, ..., -0.03712629,
        -0.08278388, -0.01702061]], dtype=float32)

In [53]:
import io

# # Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

 # Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
   if num == 0:
      continue # skip padding token
   vec = embed_weights[num]
   out_m.write(word + "\n") # write words to file
   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

 # Download files locally to upload to Embedding Projector
try:
  from google.colab import files
except ImportError:
   pass
else:
   files.download("embedding_vectors.tsv")
   files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
###https://jalammar.github.io/illustrated-word2vec/

####https://www.tensorflow.org/text/tutorials/word2vec

In [55]:
## RNN



##RNN



https://www.youtube.com/watch?v=SEnXr6v2ifU

https://karpathy.github.io/2015/05/21/rnn-effectiveness/

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

LSTM =long short term memory



Bir recurrent neural netwokün yapısı aşağıdaki gibidir,



Input -> Text Vectorization Layer -> Embedding Layer -> Tensor olarak tanımlanır -> Recurrent Katmanları (LSTM,GRU,Bidrectional)-> Dense Katmanı-> Output

In [56]:
from tensorflow.keras import layers

inputs = layers.Input(shape =(1,), dtype="string")
x= text_vectorizer(inputs)
x=embedding(x)
print(x.shape)
x= layers.LSTM(64, return_sequences=True)(x)
print(x.shape)
x= layers.LSTM(64)(x)
print(x.shape)
x=layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation ="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name ="model_2_LSTM")

##https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [57]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
model_2.summary()


Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [59]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20240310-160308
Epoch 1/5
215/215 [==============================] - 13s 42ms/step - loss: 0.2220 - accuracy: 0.9242 - val_loss: 0.5473 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 8s 39ms/step - loss: 0.1634 - accuracy: 0.9402 - val_loss: 0.5996 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 7s 34ms/step - loss: 0.1278 - accuracy: 0.9523 - val_loss: 0.7651 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 8s 37ms/step - loss: 0.1070 - accuracy: 0.9587 - val_loss: 0.9182 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 7s 34ms/step - loss: 0.0802 - accuracy: 0.9660 - val_loss: 0.9527 - val_accuracy: 0.7756


In [60]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


((762, 1),
 array([[4.8497897e-03],
        [6.1785340e-01],
        [9.9982816e-01],
        [5.0871637e-02],
        [3.9806855e-05],
        [9.9908423e-01],
        [9.2861110e-01],
        [9.9993747e-01],
        [9.9987245e-01],
        [4.2263851e-01]], dtype=float32))

In [61]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7794121823201254,
 'recall': 0.7755905511811023,
 'f1': 0.7726614113025507}

In [63]:
compare_baseline_to_new_results(baseline_results, model_2_results)


Baseline accuracy: 79.27, New accuracy: 77.56, Difference: -1.71
Baseline precision: 0.81, New precision: 0.78, Difference: -0.03
Baseline recall: 0.79, New recall: 0.78, Difference: -0.02
Baseline f1: 0.79, New f1: 0.77, Difference: -0.01


##GRU

LSTM'ye benzer ancak daha az parametresi vardır.

https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be

In [64]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x= layers.GRU(64, return_sequences=True)(x)
print(x.shape)
x=layers.LSTM(42, return_sequences=True)(x)
print(x.shape)
x= layers.GRU(99)(x)
print(x.shape)
x= layers.Dense(64, activation="relu")(x)
print(x.shape)
x = layers.Dense(1, activation= "sigmoid")(x)
outputs= layers.Dense(1, activation="sigmoid")(x)
model_3= tf.keras.Model(inputs, outputs, name="model_3_GRU")

(None, 15, 128)
(None, 15, 64)
(None, 15, 42)
(None, 99)
(None, 64)


In [65]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 lstm_2 (LSTM)               (None, 15, 42)            17976     
                                                                 
 gru_1 (GRU)                 (None, 99)                42471     
                                                       

In [66]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x= layers.GRU(64, return_sequences=True)(x)
print(x.shape)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(1, activation= "sigmoid")(x)
outputs= layers.Dense(1, activation="sigmoid")(x)
model_3= tf.keras.Model(inputs, outputs, name="model_3_GRU")

(None, 15, 128)
(None, 15, 64)


In [67]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d_1  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_6 (Dense)             (None, 1)                 

In [68]:
model_3.compile

<bound method Model.compile of <keras.src.engine.functional.Functional object at 0x787560ffa410>>

In [69]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [70]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20240310-160356
Epoch 1/5
215/215 [==============================] - 9s 31ms/step - loss: 0.5008 - accuracy: 0.8759 - val_loss: 0.5801 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 7s 31ms/step - loss: 0.4388 - accuracy: 0.9566 - val_loss: 0.5801 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 6s 28ms/step - loss: 0.4023 - accuracy: 0.9647 - val_loss: 0.5635 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 7s 32ms/step - loss: 0.3708 - accuracy: 0.9705 - val_loss: 0.5603 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.3435 - accuracy: 0.9730 - val_loss: 0.5486 - val_accuracy: 0.7717


In [71]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


((762, 1),
 array([[0.19096415],
        [0.65275   ],
        [0.6534315 ],
        [0.18941289],
        [0.18940239],
        [0.6534309 ],
        [0.6528754 ],
        [0.6534315 ],
        [0.65343153],
        [0.34743318]], dtype=float32))

In [72]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [73]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.7720646185150423,
 'recall': 0.7716535433070866,
 'f1': 0.7702930138828643}

In [74]:
compare_baseline_to_new_results(baseline_results, model_3_results)


Baseline accuracy: 79.27, New accuracy: 77.17, Difference: -2.10
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.02
Baseline f1: 0.79, New f1: 0.77, Difference: -0.02


## **BIDIRECTIONAL NEURAL NETWORK**


Standart bir RNN, bir diziyi soldan sağa doğru işleyecektir, oysa bir çift yönlü RNN, diziyi soldan sağa doğru işleyecek ve ardından sağdan sola doğru işleyecektir.

Kavramsal olarak, bu, bir cümleyi normal moda (soldan sağa) ilk kez okurken, bir nedenle anlamadığınızı varsayarsanız, kelimelerin üzerinden geri dönüp tekrar geçmek gibi düşünülebilir.

In [77]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x= layers.Bidirectional(layers.LSTM(64, return_sequences= True))(x)
x= layers.Bidirectional(layers.GRU(64))(x)
x=layers.Dense(1, activation="sigmoid")(x)
outputs= layers.Dense(1, activation="sigmoid")(x)
model_4= tf.keras.Model(inputs, outputs, name="model_4_bidirectional")


In [78]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 15, 128)           98816     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                             

In [79]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [80]:
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240310-162420
Epoch 1/5
215/215 [==============================] - 31s 84ms/step - loss: 0.5079 - accuracy: 0.9348 - val_loss: 0.5991 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 12s 54ms/step - loss: 0.4460 - accuracy: 0.9743 - val_loss: 0.5680 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 11s 53ms/step - loss: 0.4040 - accuracy: 0.9752 - val_loss: 0.5544 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 12s 54ms/step - loss: 0.3691 - accuracy: 0.9753 - val_loss: 0.5435 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 12s 57ms/step - loss: 0.3384 - accuracy: 0.9759 - val_loss: 0.5432 - val_accuracy: 0.7703


In [83]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 4s 17ms/step


array([[0.30228215],
       [0.8040697 ],
       [0.80407035],
       [0.30221638],
       [0.30221206],
       [0.8040702 ],
       [0.80403554],
       [0.80407035],
       [0.80407035],
       [0.3538133 ]], dtype=float32)

In [85]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [86]:
model_4_results = calculate_results(y_true=val_labels, y_pred=model_4_preds)
model_4_results

{'accuracy': 77.03412073490814,
 'precision': 0.7709806133638261,
 'recall': 0.7703412073490814,
 'f1': 0.7688058109983439}

In [ ]:
import tensorflow.keras import

## **Metin için CNN**


Inputs (text) -> Tokenization -> Embedding -> Layers -> Outputs (class probabilities)

LSTM veya GRU hücreleri yerine Conv1D veya GlobalMaxPool1D katmanlarını kullanacağız

In [87]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu") # convolve over target sequence 5 words at a time
conv_1d_output = conv_1d(embedding_test) # pass embedding through 1D convolutional layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important features
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [88]:
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]


(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.00676629,  0.02023377,  0.00600205, ...,  0.01619705,
          -0.01653761, -0.01139553],
         [ 0.02742362,  0.00060547,  0.00253709, ..., -0.08439735,
          -0.05442083, -0.07131065],
         [ 0.03560755,  0.03399605,  0.01665656, ...,  0.0171346 ,
          -0.02413922, -0.03705096],
         ...,
         [-0.03428087, -0.00866521,  0.03300232, ...,  0.0107111 ,
           0.00712816, -0.02522406],
         [-0.03428087, -0.00866521,  0.03300232, ...,  0.0107111 ,
           0.00712816, -0.02522406],
         [-0.03428087, -0.00866521,  0.03300232, ...,  0.0107111 ,
           0.00712816, -0.02522406]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.01448993, 0.        , 0.02314391, 0.        , 0.09729609,
          0.04068511, 0.01469259, 0.04872901, 0.        , 0.        ,
          0.04528856, 0.09823339, 0.        , 0.01994472, 0.05199602,
          0.        , 0.

In [89]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our 1D convolution model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 32)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_11 (Dense)            (None, 1)              

In [90]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240310-164102
Epoch 1/5
215/215 [==============================] - 7s 29ms/step - loss: 0.5646 - accuracy: 0.7160 - val_loss: 0.4755 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3369 - accuracy: 0.8635 - val_loss: 0.4767 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2060 - accuracy: 0.9235 - val_loss: 0.5382 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.1318 - accuracy: 0.9578 - val_loss: 0.6029 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0936 - accuracy: 0.9685 - val_loss: 0.6554 - val_accuracy: 0.7913


In [91]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.5623362 ],
       [0.7088148 ],
       [0.9996931 ],
       [0.08684821],
       [0.02119648],
       [0.9886171 ],
       [0.89589304],
       [0.9978046 ],
       [0.9984166 ],
       [0.06607866]], dtype=float32)

In [92]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [93]:
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 79.13385826771653,
 'precision': 0.793352563602817,
 'recall': 0.7913385826771654,
 'f1': 0.7895046541197648}

In [94]:
compare_baseline_to_new_results(baseline_results, model_5_results)


Baseline accuracy: 79.27, New accuracy: 79.13, Difference: -0.13
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.00
Baseline f1: 0.79, New f1: 0.79, Difference: 0.00


In [101]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.MaxPooling1D(pool_size=2)(x)  # MaxPooling1D kullanımı
x = layers.GlobalMaxPooling1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_5.summary()



Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 max_pooling1d (MaxPooling1  (None, 5, 64)             0         
 D)                                                              
                                                                 
 global_max_pooling1d_4 (Gl  (None, 64)             

In [103]:
model_5_history = model_5.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences, val_labels), callbacks=[create_tensorboard_callback(SAVE_DIR,"Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240310-171807
Epoch 1/5
215/215 [==============================] - 6s 21ms/step - loss: 0.1554 - accuracy: 0.9550 - val_loss: 0.7378 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1010 - accuracy: 0.9691 - val_loss: 0.8867 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0805 - accuracy: 0.9726 - val_loss: 0.9361 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0697 - accuracy: 0.9742 - val_loss: 1.0049 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0608 - accuracy: 0.9765 - val_loss: 1.0744 - val_accuracy: 0.7717


In [104]:
model_5_pred= model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.5623362 ],
       [0.7088148 ],
       [0.9996931 ],
       [0.08684821],
       [0.02119648],
       [0.9886171 ],
       [0.89589304],
       [0.9978046 ],
       [0.9984166 ],
       [0.06607866]], dtype=float32)

In [105]:
model_5_results= calculate_results(y_true= val_labels, y_pred=model_5_preds)

model_5_results

{'accuracy': 79.13385826771653,
 'precision': 0.793352563602817,
 'recall': 0.7913385826771654,
 'f1': 0.7895046541197648}